# Make a Simulation Cell with Aqueous Solution
Create a cell with a single solute atom

In [1]:
from ase.optimize import LBFGS
from ase.build import molecule
from ase.io import read
from ase import Atoms, units
from subprocess import run
from pathlib import Path
from random import randint
import numpy as np

Configuration

In [2]:
distance_tolerance = 2  # Ensure they do not get too close
density = 1.1  # In g/cm^3, based only on the water
solute = 'CH4'
solvent = 'H2O'
num_solvent = 32
seed = randint(0, 256)

## Make the solution with Packmol
Give it PBC and ensure we have just enough spacing between molecules

In [3]:
solvent_atoms = molecule(solvent)
solute_atoms = molecule(solute)

Write the structure to disk in PDB format

In [4]:
solvent_atoms.write('solvent.pdb')

In [5]:
solute_atoms.write('solute.pdb')

Compute the density based on the solvent

In [6]:
total_mass = solvent_atoms.get_masses().sum() * num_solvent / units.mol # g

In [7]:
volume = (total_mass / units.mol) / density * (units.kg / 1000)  # cm^3
volume /= 1e-24  # A^3

In [8]:
side_length = np.power(volume, 1./3)
print(f'Building a box with side lengths of {side_length:.2f} A')

Building a box with side lengths of 9.55 A


Assemble the input file

In [9]:
middle = side_length / 2
packmol_inp = f'''
seed {seed}
tolerance {distance_tolerance:.1f}
output cell.pdb
structure solvent.pdb
  number {num_solvent}
  inside cube 0. 0. 0. {side_length:.2f}
end structure
structure solute.pdb
   number 1
   center 
   fixed {middle:.2f} {middle:.2f} {middle:.2f} 0. 0. 0.
end structure 
'''

In [10]:
Path('packmol.inp').write_text(packmol_inp)

208

In [11]:
with Path('packmol.inp').open() as fp:
    run('packmol', stdin=fp, capture_output=True)

In [12]:
cell = read('cell.pdb')

Make it periodic and add a buffer around the side for PBC

In [13]:
cell.pbc = True

In [14]:
cell.cell = [side_length + distance_tolerance / 2] * 3

Save it to disk

In [15]:
Path('initial-geometries').mkdir(exist_ok=True)
cell.write(f'initial-geometries/packmol-{solute}-in-{solvent}={num_solvent}-seed={seed}.vasp', sort=True)